# Nimbus prediction notebook 

In [1]:
# import required packages
import warnings
warnings.simplefilter("ignore")
import os
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from nimbus_inference.nimbus import Nimbus, prep_naming_convention
from nimbus_inference.utils import MultiplexDataset
from alpineer import io_utils
#from ark.utils import example_dataset
from nimbus_inference.viewer_widget import NimbusViewer

## 0: Set root directory and download example dataset
Here we are using the example data located in `/data/example_dataset/input_data`. To modify this notebook to run using your own data, simply change `base_dir` to point to your own sub-directory within the data folder. Set `base_dir`, the path to all of your imaging data (i.e. multiplexed images and segmentation masks). Subdirectory `nimbus_output` will contain all of the data generated by this notebook. In the following, we expect this folder structure, with `fov_1` and `fov_2` either being folders of individual channel images or `.ome.tiff` files that contain all channels in a single file.
```bash
|-- base_dir
|   |-- image_data
|   |   |-- fov_1
|   |   |-- fov_2
|   |-- segmentation
|   |   |-- deepcell_output
|   |-- nimbus_output
```

In [2]:
# set up the base directory
base_dir = os.path.normpath("/Volumes/Gut_Project/pheno_benchmark/IMMUcan")
output_dir = os.path.normpath("/Users/margotchazotte/Documents/uni/PhD/pheno_benchmark/nimbus/IMMUcan")


If you would like to test Nimbus with an example dataset, run the cell below. It will download a dataset consisting of 10 FOVs with 22 channels. You may find more information about the example dataset in the [ark-analysis README](https://github.com/angelolab/ark-analysis/blob/bc6685050dfbef4607874fbbadebd4289251c173/README.md#example-dataset). If you want to use your own data, skip the cell below


## 1: set file paths and parameters

### All data, images, files, etc. must be placed in the 'data' directory, and referenced via '../data/path_to_your_data'


In [46]:
# set up file paths
tiff_dir = os.path.join(base_dir, "raw_images/unstacked")
deepcell_output_dir = os.path.join(base_dir, "segmentation")
nimbus_output_dir = os.path.join(output_dir, "it5/nimbus_output")

# Create nimbus output directory
os.makedirs(nimbus_output_dir, exist_ok=True)

# Check if paths exist
io_utils.validate_paths([base_dir, tiff_dir, deepcell_output_dir, nimbus_output_dir])

## 2: Set up input paths and the naming convention for the segmentation data
Store names of channels to exclude in the list below. Either predict all FOVs or specify manually the ones you want to apply Nimbus on.

In [47]:
# define the channels to include
#manually
include_channels = [
    "MPO", "SMA", "CD16", "CD38", "HLADR", "CD27", "CD15",
    "CD45RA", "CD163", "B2M", "CD20", "CD68", "Ido1", "CD3", "LAG3", 
    "CD11c", "PD1", "PDGFRb", "CD7", "GrzB", "PDL1", "TCF7", "CD45RO",
    "FOXP3", "ICOS", "CD8a", "CarbonicAnhydrase", "CD33", "Ki67", 
    "VISTA", "CD40", "CD4", "CD14", "Ecad", "CD303", "CD206", 
    "cleavedPARP"
]
#or read in from txt file
#with open(os.path.join(base_dir, "markers.txt"), "r") as file:
#    include_channels = file.readlines()
#    include_channels = [line.strip() for line in include_channels]  # remove newline characters

# either get all fovs in the folder...
fov_names = os.listdir(tiff_dir)
# ... or optionally, select a specific set of fovs manually
# fovs = ["fov0", "fov1"]

# make sure to filter paths out that don't lead to FoVs, e.g. .DS_Store files.
fov_names = [fov_name for fov_name in fov_names if not fov_name.startswith(".")] 

# construct paths for fovs
fov_paths = [os.path.join(tiff_dir, fov_name) for fov_name in fov_names]

In [37]:
fov_paths , fov_names

(['/Volumes/Gut_Project/pheno_benchmark/IMMUcan/raw_images/unstacked/IMMUcan_Batch20211215_10074353-SPECT-VAR-TIS-UNST-03_002',
  '/Volumes/Gut_Project/pheno_benchmark/IMMUcan/raw_images/unstacked/IMMUcan_Batch20200528_S-200120-00002_006',
  '/Volumes/Gut_Project/pheno_benchmark/IMMUcan/raw_images/unstacked/IMMUcan_Batch20211215_10082071-SPECT-VAR-TIS-01-IMC-01_003',
  '/Volumes/Gut_Project/pheno_benchmark/IMMUcan/raw_images/unstacked/IMMUcan_2022_WFLOW_10061074-SPECT-VAR-TIS-01-IMC-01_001',
  '/Volumes/Gut_Project/pheno_benchmark/IMMUcan/raw_images/unstacked/IMMUcan_2022_WFLOW_10034294-GI-VAR-TIS-UNST-03_004',
  '/Volumes/Gut_Project/pheno_benchmark/IMMUcan/raw_images/unstacked/IMMUcan_Batch20210304_10060017-SPECT-VAR-TIS-01-IMC-01_v2_003',
  '/Volumes/Gut_Project/pheno_benchmark/IMMUcan/raw_images/unstacked/IMMUcan_Batch20220609_10089094-SPECT-VAR-TIS-01-IMC-01_001',
  '/Volumes/Gut_Project/pheno_benchmark/IMMUcan/raw_images/unstacked/IMMUcan_Batch20210701_S-210510-00002_005',
  '/Vo

Define the naming convention for the segmentation data in function `segmentation_naming_convention`, that maps the `fov_name` to the path of the associated segmentation output. The below function `prep_deepcell_naming_convention` assumes that all segmentation outputs are stored in one folder, with the `fov_name` as the prefix and `_whole_cell.tiff` as the suffix, as shown below in the visualization of the folder structure. If this does not apply to your data, you have to define a function `segmentation_naming_convention` that takes an element from `fov_paths` and returns a valid path to the segmentation label map you want to use for that fov.

```
|-- base_dir
|   |-- image_data
|   |   |-- fov_1
|   |   |-- fov_2
|   |-- segmentation
|   |   |-- deepcell_output
|   |   |   |-- fov_1_whole_cell.tiff
|   |   |   |-- fov_2_whole_cell.tiff
|   |-- nimbus_output
```

In [48]:
# adjust definition such as input of fov_paths[i] returns the path to the matching segmentation data
def prep_naming_convention(deepcell_output_dir):
    """Prepares the naming convention for the segmentation data produced with the DeepCell library.

    Args:
        deepcell_output_dir (str): path to directory where segmentation data is saved
    Returns:
        function: function that returns the path to the
            segmentation data for a given fov
    """

    def segmentation_naming_convention(fov_path):
        """Prepares the path to the segmentation data for a given fov

        Args:
            fov_path (str): path to fov
        Returns:
            str: paths to segmentation fovs
        """
        fov_name = os.path.basename(fov_path)#.replace(".ome.tiff", "")
        return os.path.join(deepcell_output_dir, fov_name + ".tiff") # + ".tiff"

    return segmentation_naming_convention

In [49]:
# Prepare segmentation naming convention that maps a fov_path to the according segmentation label map
segmentation_naming_convention = prep_naming_convention(deepcell_output_dir)

# test segmentation_naming_convention
if os.path.exists(segmentation_naming_convention(fov_paths[0])):
    print("Segmentation data exists for fov 0 and naming convention is correct")
else:
    print("Segmentation data does not exist for fov 0 or naming convention is incorrect")

Segmentation data exists for fov 0 and naming convention is correct


Next we will use the `MultiplexDataset` class to abstract away differences in data representation. The class takes `fov_paths`, `segmentation_naming_convention` and a `suffix` and provides methods `.get_channel(fov, channel)` and `.get_segmentation(fov)` to access the data. The `suffix` is used to filter out files that do not end with the specified suffix. When you use `.ome.tiff` files make sure to set the suffix to `.ome.tiff`, otherwise the ViewerWidget won't be able to display the images.

In [50]:
dataset = MultiplexDataset(
    fov_paths=fov_paths,
    suffix=".tiff", # or .png, .jpg, .jpeg, .tif or .ome.tiff
    include_channels=include_channels,
    segmentation_naming_convention=segmentation_naming_convention
)

All inputs are valid


## 3: Load model and initialize Nimbus application
The following code initializes the Nimbus application and loads the model checkpoint. The model was trained on a diverse set of tissues, protein markers, imaging platforms and cell types and doesn't need re-training. If you want to use the model on a machine without GPU, set `test_time_aug=False` to speed up inference. If you run it on a laptop GPU and run into out-of-memory errors, consider reducing the `batch_size` to 1 and the `input_shape` to `[512,512]`.

In [51]:
nimbus = Nimbus(
    dataset=dataset,
    output_dir=nimbus_output_dir,
    save_predictions=True,
    batch_size=4,
    test_time_aug=False,
    input_shape=[1024,1024],
    device="mps",
)

# check if all inputs are valid
nimbus.check_inputs()

All inputs are valid.


## 4: Prepare normalization dictionary 
The next step is to iterate through all the fovs and calculate the 0.999 marker expression quantile for each marker individually. This is used for normalizing the marker expressions prior to predicting marker confidence scores with our model. You can set `n_subset` to estimate the quantiles on a small subset of the data and you can set `multiprocessing=True` to speed up computation.

In [52]:
nimbus.prepare_normalization_dict(
    quantile=0.999,
    n_subset=50,
    clip_values=(0, 2),
    multiprocessing=True,
    overwrite=True
)

Iterate over fovs...


## 5: Make predictions with the model
Nimbus will iterate through your samples and store predictions and a file named `nimbus_cell_table.csv` that contains the mean-per-cell predicted marker confidence scores in the sub-directory called `nimbus_output`.

In [53]:
cell_table = nimbus.predict_fovs()

Available GPUs:  0
Predictions will be saved in /Users/margotchazotte/Documents/uni/PhD/pheno_benchmark/nimbus/IMMUcan/it5/nimbus_output
Iterating through fovs will take a while...
Predicting /Volumes/Gut_Project/pheno_benchmark/IMMUcan/raw_images/unstacked/IMMUcan_Batch20211215_10074353-SPECT-VAR-TIS-UNST-03_002...


  0%|          | 0/37 [00:00<?, ?it/s]

Checking for updated model checkpoints on HuggingFace Hub...
Using existing checkpoint: /Users/margotchazotte/anaconda3/envs/nimbus-env/lib/python3.9/site-packages/nimbus_inference/assets/V1.pt
Loaded weights from /Users/margotchazotte/anaconda3/envs/nimbus-env/lib/python3.9/site-packages/nimbus_inference/assets/V1.pt
Predicting /Volumes/Gut_Project/pheno_benchmark/IMMUcan/raw_images/unstacked/IMMUcan_Batch20200528_S-200120-00002_006...


  0%|          | 0/37 [00:00<?, ?it/s]

Predicting /Volumes/Gut_Project/pheno_benchmark/IMMUcan/raw_images/unstacked/IMMUcan_Batch20211215_10082071-SPECT-VAR-TIS-01-IMC-01_003...


...

  0%|          | 0/37 [00:00<?, ?it/s]

Predicting /Volumes/Gut_Project/pheno_benchmark/IMMUcan/raw_images/unstacked/IMMUcan_Batch20201113_10034034-THOR-VAR-TIS-01-IMC-01_002...


  0%|          | 0/37 [00:00<?, ?it/s]

Predicting /Volumes/Gut_Project/pheno_benchmark/IMMUcan/raw_images/unstacked/IMMUcan_Batch20210701_UPSTREAM_S-210510-00032_006...


  0%|          | 0/37 [00:00<?, ?it/s]

In [54]:
cell_table

,label,fov,MPO,SMA,CD16,CD38,HLADR,CD27,CD15,CD45RA,...,CD33,Ki67,VISTA,CD40,CD4,CD14,Ecad,CD303,CD206,cleavedPARP
0,1,IMMUcan_Batch20211215_10074353-SPECT-VAR-TIS-U...,0.027751,0.022470,0.023404,0.022524,0.022405,0.026915,0.035727,0.023823,...,0.070164,0.023221,0.040830,0.022594,0.022999,0.029980,0.048794,0.044323,0.022998,0.025821
1,2,IMMUcan_Batch20211215_10074353-SPECT-VAR-TIS-U...,0.083230,0.145367,0.094958,0.085375,0.082007,0.112736,0.104315,0.075584,...,0.082835,0.152507,0.054115,0.115162,0.085805,0.123358,0.249644,0.083003,0.079307,0.096320
2,3,IMMUcan_Batch20211215_10074353-SPECT-VAR-TIS-U...,0.044952,0.036882,0.034170,0.033201,0.036347,0.037821,0.099035,0.034497,...,0.030034,0.090537,0.034924,0.034835,0.036648,0.035797,0.277486,0.030377,0.034804,0.036139
3,4,IMMUcan_Batch20211215_10074353-SPECT-VAR-TIS-U...,0.024287,0.026124,0.023799,0.023822,0.023708,0.023518,0.026510,0.023405,...,0.026171,0.025185,0.035607,0.023802,0.023469,0.023916,0.069152,0.022917,0.023890,0.024467
4,5,IMMUcan_Batch20211215_10074353-SPECT-VAR-TIS-U...,0.024870,0.036165,0.155621,0.026002,0.049094,0.126799,0.025698,0.026735,...,0.047395,0.026355,0.070118,0.077922,0.195910,0.542450,0.025991,0.027749,0.034618,0.025315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460281,3138,IMMUcan_Batch20210701_UPSTREAM_S-210510-00032_006,0.028373,0.025865,0.026107,0.024747,0.026059,0.024748,0.025474,0.023621,...,0.023153,0.027027,0.023991,0.024692,0.023331,0.024342,0.031547,0.025388,0.026108,0.024397
460282,3139,IMMUcan_Batch20210701_UPSTREAM_S-210510-00032_006,0.028545,0.038045,0.034535,0.038155,0.036201,0.040364,0.032663,0.037424,...,0.030855,0.386676,0.042323,0.040135,0.042208,0.054197,0.083276,0.054894,0.035522,0.036753
460283,3140,IMMUcan_Batch20210701_UPSTREAM_S-210510-00032_006,0.034058,0.028803,0.033531,0.034573,0.034858,0.029796,0.035711,0.067129,...,0.044787,0.035509,0.029531,0.039089,0.047167,0.047117,0.659641,0.063684,0.035378,0.047748
460284,3141,IMMUcan_Batch20210701_UPSTREAM_S-210510-00032_006,0.018196,0.017695,0.018191,0.018547,0.017508,0.018398,0.018179,0.079774,...,0.020684,0.016494,0.023931,0.020587,0.027222,0.026192,0.290573,0.042359,0.017575,0.022773


In [55]:
cell_table.to_csv(os.path.join(nimbus_output_dir, "nimbus_cell_table.csv"), index=False)

## 6: View multiplexed channels and Nimbus predictions side-by-side
Select an FOV and one marker image per channel to inspect the imaging data and associated Nimbus predictions